# Exercice 3   

**Question 1**: Modelisation du problème de plus court chemin comme problème du flot de coût minimum

$G=(V,E)$, source: $s \in V$, puit: $t \in V$

$a(u,v)$: cout pour passer de u à v, $f(u,v)=1$ si $(u,v) \in $ {plus court chemin} 

Programme linéaire:
\begin{align*}
\min&\sum_{(u,v)\in E} a(u,v)f(u,v)&\\
s.c.& \sum_{(u,v)\in E}f(u,v) = \sum_{(v,w)\in E}f(v,w) \quad \forall v \in V \setminus \{s, t\}& \text{, contrainte de conservation du flux}\\
    & \sum_{s,v \in E} f(s,v) >= 1 &\text{ , contrainte de flux sortant de la source}\\
    &f(u,v)\in (0,1) \quad \forall (u,v) \in E&
\end{align*}

**Question 2:** Application aux exemples

In [1]:
from gurobipy import *

In [2]:
couts_i1 = [{
        ('a','b'): 4,
        ('a','c'): 5,
        ('b','c'): 2,
        ('b','d'): 1,
        ('b','e'): 2,
        ('b','f'): 7,
        ('c','d'): 5,
        ('c','e'): 2,
        ('d','f'): 3,
        ('e','f'): 5
    },
    {
        ('a','b'): 3,
        ('a','c'): 1,
        ('b','c'): 1,
        ('b','d'): 4,
        ('b','e'): 2,
        ('b','f'): 5,
        ('c','d'): 1,
        ('c','e'): 7,
        ('d','f'): 2,
        ('e','f'): 2
    }
]

couts_i2 = [
    {
        ('a','b'): 5,
        ('a','c'): 10,
        ('a','d'): 2,
        ('b','c'): 4,
        ('b','d'): 1,
        ('b','e'): 4,
        ('c','e'): 3,
        ('c','f'): 1,
        ('d','c'): 1,
        ('d','f'): 3,
        ('e','g'): 1,
        ('f','g'): 1
    },
    {
        ('a','b'): 3,
        ('a','c'): 4,
        ('a','d'): 6,
        ('b','c'): 2,
        ('b','d'): 3,
        ('b','e'): 6,
        ('c','e'): 1,
        ('c','f'): 2,
        ('d','c'): 4,
        ('d','f'): 5,
        ('e','g'): 1,
        ('f','g'): 1
    }
]



In [3]:
def minFlow(couts):
    # trouver noeuds
    noeuds = set()
    for arc in couts:
        noeuds.update(arc)
    noeuds = list(noeuds)

    source = min(noeuds)
    puits = max(noeuds)

    model = Model()
    model.params.LogToConsole = False

    f = model.addVars(couts.keys(), vtype = GRB.BINARY, name='f')
    model.setObjective(quicksum([f[i,j] * cout for (i,j),cout in couts.items()]), GRB.MINIMIZE)

    # contrainte de conservation du flux
    for v in noeuds:
        if v == source or v == puits:
            continue
        flux_entrant = quicksum([f[i,v] for i,j in couts if j == v])
        flux_sortant = quicksum([f[v,j] for i,j in couts if i == v])
        model.addConstr(flux_entrant == flux_sortant)

    #for arc, cout in couts.items():
        #model.addConstr(f[arc] <= cout )    # contrainte de capacité
        #model.addConstr(f[arc] >= 0)        # contrainte de non-negativité


    model.addConstr(quicksum([f[source, j] for i,j in couts if i == source]) >= 1)


    model.optimize()
    objets_selectiones = [(i,j) for i,j in couts if f[i,j].X > 0.5]
    print('Chemin: ', objets_selectiones,', coût: ', int(model.ObjVal))

In [4]:
for i, couts_instances in enumerate([couts_i1, couts_i2]):
    print(f"Plus court chemin de l'instance {i} : ")
    for j,couts in enumerate(couts_instances):
        print(f"Scénario {j}:")
        minFlow(couts)


Plus court chemin de l'instance 0 : 
Scénario 0:
Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-18
Chemin:  [('a', 'b'), ('b', 'd'), ('d', 'f')] , coût:  8
Scénario 1:
Chemin:  [('a', 'c'), ('c', 'd'), ('d', 'f')] , coût:  4
Plus court chemin de l'instance 1 : 
Scénario 0:
Chemin:  [('a', 'd'), ('c', 'f'), ('d', 'c'), ('f', 'g')] , coût:  5
Scénario 1:
Chemin:  [('a', 'c'), ('c', 'e'), ('e', 'g')] , coût:  6


**Question 3:**

## maxmin
Programme linéaire pour $n$ scénarios:
\begin{align*}
\min\quad   & t&&\\
s.c.\quad   &t\ge\sum_{(u,v)\in E} a_i(u,v)f(u,v)&i\in\{1,\ldots,n\}\\
            & \sum_{(u,v)\in E}f(u,v) = \sum_{(v,w)\in E}f(v,w) \quad \forall v \in V \setminus \{s, t\}& \text{, contrainte de conservation du flux}\\
            & \sum_{s,v \in E} f(s,v) >= 1 &\text{ , contrainte de flux sortant de la source}&\\
            &f(u,v)\in \{0,1\} \quad \forall (u,v) \in E&
\end{align*}

In [5]:
def minFlowMaxMin(valeurs_scenarios):
    # trouver noeuds
    # on suppose que tous les scénarios ont les mêmes arcs


    noeuds = set()
    for arc in valeurs_scenarios[0]:
        noeuds.update(arc)
    noeuds = list(noeuds)

    source = min(noeuds)
    puits = max(noeuds)

    model = Model()
    model.params.LogToConsole = False

    f = model.addVars(valeurs_scenarios[0].keys(), vtype = GRB.BINARY, name='f')
    t = model.addVar(vtype = GRB.INTEGER)


    model.setObjective(t, GRB.MINIMIZE)
   
    # contrainte de t
    for couts in valeurs_scenarios:
        model.addConstr(t >= quicksum([f[i,j] * cout for (i,j),cout in couts.items()]))

    # contrainte de conservation du flux
    for v in noeuds:
        if v == source or v == puits:
            continue
        flux_entrant = quicksum([f[i,v] for i,j in couts if j == v])
        flux_sortant = quicksum([f[v,j] for i,j in couts if i == v])
        model.addConstr(flux_entrant == flux_sortant)

    #for arc, cout in couts.items():
        #model.addConstr(f[arc] <= cout )    # contrainte de capacité
        #model.addConstr(f[arc] >= 0)        # contrainte de non-negativité


    model.addConstr(quicksum([f[source, j] for i,j in couts if i == source]) >= 1)


    model.optimize()

    objets_selectiones = [(i,j) for i,j in couts if f[i,j].X > 0.5]

    vecteur_z = []
    for v_s in valeurs_scenarios:
        vecteur_z.append(sum([v_s[i,j] for (i,j) in objets_selectiones]))
    cout_chemin = int(model.ObjVal)

    return {'vecteur_z':vecteur_z, 'objets': objets_selectiones, 'profit': cout_chemin}

In [6]:
for i, couts_instances in enumerate([couts_i1, couts_i2]):
    print(f"Plus court chemin de l'instance {i} : ")
    gurobi_solution = minFlowMaxMin(couts_instances)
    print('Chemin: ', gurobi_solution['objets'], 'cout: ', gurobi_solution['profit'], ' vecteur_z: ', gurobi_solution['vecteur_z'])



Plus court chemin de l'instance 0 : 
Chemin:  [('a', 'b'), ('b', 'd'), ('d', 'f')] cout:  9  vecteur_z:  [8, 9]
Plus court chemin de l'instance 1 : 
Chemin:  [('a', 'b'), ('b', 'e'), ('e', 'g')] cout:  10  vecteur_z:  [10, 10]


## minmax regret
Programme linéaire pour $n$ scénarios:
\begin{align*}
\min\quad   & t&&\\
s.c.\quad   &t\ge\sum_{(u,v)\in E} \bigl(a_i(u,v)f(u,v)\bigl)-z_i^*&i\in\{1,\ldots,n\}\\
            & \sum_{(u,v)\in E}f(u,v) = \sum_{(v,w)\in E}f(v,w) \quad \forall v \in V \setminus \{s, t\}& \text{, contrainte de conservation du flux}\\
            & \sum_{s,v \in E} f(s,v) >= 1 &\text{ , contrainte de flux sortant de la source}&\\
            &f(u,v)\in \{0,1\} \quad \forall (u,v) \in E&
\end{align*}

In [7]:
def minFlowMinRegret(valeurs_scenarios):

     # trouver les meilleures solutions pour chaque scénario
    z_etoile = []
    for v_s in valeurs_scenarios:
        gurobi_solution = minFlowMaxMin([v_s]) # minFlowMaxMin() avec un scénario est égal à optimiser le problème sad
        z_etoile.append(gurobi_solution['profit'])

    print("z*=", z_etoile)

    # trouver noeuds
    # on suppose que tous les scénarios ont les mêmes arcs
    noeuds = set()
    for arc in valeurs_scenarios[0]:
        noeuds.update(arc)
    noeuds = list(noeuds)

    source = min(noeuds)
    puits = max(noeuds)

    model = Model()
    model.params.LogToConsole = False

    f = model.addVars(valeurs_scenarios[0].keys(), vtype = GRB.BINARY, name='f')
    t = model.addVar(vtype = GRB.INTEGER)


    model.setObjective(t, GRB.MINIMIZE)
   
    # contrainte de t
    for i,couts in enumerate(valeurs_scenarios):
        model.addConstr(t >= quicksum([f[i,j] * cout for (i,j),cout in couts.items()]) - z_etoile[i])

    # contrainte de conservation du flux
    for v in noeuds:
        if v == source or v == puits:
            continue
        flux_entrant = quicksum([f[i,v] for i,j in valeurs_scenarios[0] if j == v])
        flux_sortant = quicksum([f[v,j] for i,j in valeurs_scenarios[0] if i == v])
        model.addConstr(flux_entrant == flux_sortant)

    #for arc, cout in couts.items():
        #model.addConstr(f[arc] <= cout )    # contrainte de capacité
        #model.addConstr(f[arc] >= 0)        # contrainte de non-negativité


    model.addConstr(quicksum([f[source, j] for i,j in couts if i == source]) >= 1)


    model.optimize()

    objets_selectiones = [(i,j) for i,j in couts if f[i,j].X > 0.5]

    vecteur_z = []
    for v_s in valeurs_scenarios:
        vecteur_z.append(sum([v_s[i,j] for (i,j) in objets_selectiones]))

    min_regret = int(model.ObjVal)

    return {'vecteur_z':vecteur_z, 'objets': objets_selectiones, 'profit': min_regret}

In [8]:
for i, couts_instances in enumerate([couts_i1, couts_i2]):
    print(f"Plus court chemin de l'instance {i} : ")
    gurobi_solution = minFlowMinRegret(couts_instances)
    print('Chemin: ', gurobi_solution['objets'], ', min_regret: ', gurobi_solution['profit'], ', vecteur_z: ', gurobi_solution['vecteur_z'])
    print("--------------------------------------------------------------------------------------------------\n")

Plus court chemin de l'instance 0 : 
z*= [8, 4]
Chemin:  [('a', 'b'), ('b', 'e'), ('e', 'f')] , min_regret:  3 , vecteur_z:  [11, 7]
--------------------------------------------------------------------------------------------------

Plus court chemin de l'instance 1 : 
z*= [5, 6]
Chemin:  [('a', 'b'), ('b', 'e'), ('e', 'g')] , min_regret:  5 , vecteur_z:  [10, 10]
--------------------------------------------------------------------------------------------------



## minOWA

\begin{align*}
\min            &\sum_{k=1}^n w'_k \left( k \cdot r_k + \sum_{i=1}^n b_{ik} \right) \\
\text{s.c.}     & \quad r_k + b_{ik} \geq z_i(x) \quad i = 1,\ldots,n\\
                &\quad x \in X\\
                & \quad b_{ik} \geq 0, \quad i = 1, \dots, n \\
\end{align*}

In [15]:
def minOWA(valeurs_scenarios,w):

    model = Model()
    model.params.LogToConsole = False
    # trouver noeuds
    # on suppose que tous les scénarios ont les mêmes arcs
    noeuds = set()
    for arc in valeurs_scenarios[0]:
        noeuds.update(arc)
    noeuds = list(noeuds)

    source = min(noeuds)
    puits = max(noeuds)

    n_scenarios = len(valeurs_scenarios)

    f = model.addVars(valeurs_scenarios[0].keys(), vtype = GRB.BINARY, name='f')

    #w = [i+1 for i in range(n_scenarios)][::-1]
    w_apostrophe = [w[k] - w[k+1] for k in range(len(w)-1)]
    w_apostrophe.append(w[-1])
   # print(w_apostrophe)
    rk = model.addVars(n_scenarios,vtype=GRB.CONTINUOUS, name='rk')
    bik = []
    
    #n**2 variables b 
    for k in range(n_scenarios):
        bik.append(model.addVars(n_scenarios,vtype=GRB.CONTINUOUS, lb=0, name=f'bik {k}'))
    objectif = quicksum([w_apostrophe[k]*((k+1)*rk[k]+quicksum([bik[k][i] for i in range(n_scenarios)])) for k in range(n_scenarios)])
    model.update()
   # print(objectif)
    model.setObjective(objectif, GRB.MINIMIZE)
    
    # calculer vecteur z
    z = []
    for couts in valeurs_scenarios:
        z.append(quicksum([f[i,j] * cout for (i,j),cout in couts.items()]))
    model.update()
    #print("vecteur z")
    #print(z)
    #print("----------")
    for k in range(n_scenarios):
        for i in range(n_scenarios):
            model.addConstr(rk[k]+bik[k][i]>=z[i]) # premiere contrainte

    # contrainte de conservation du flux
    for v in noeuds:
        if v == source or v == puits:
            continue
        flux_entrant = quicksum([f[i,v] for i,j in valeurs_scenarios[0] if j == v])
        flux_sortant = quicksum([f[v,j] for i,j in valeurs_scenarios[0] if i == v])
        c_cf = flux_entrant == flux_sortant
        model.update()
        #print(c_cf)
        model.addConstr(c_cf)

    # contrainte de flux sortant de la source
    c_fs = quicksum([f[source, j] for i,j in valeurs_scenarios[0] if i == source]) >= 1
    model.update()
   # print(c_fs)
    model.addConstr(c_fs)
    model.optimize()
    
    objets_selectiones = [(i,j) for i,j in valeurs_scenarios[1] if f[i,j].X > 0.5]
   # for b_array in bik:
        #print(b_array)

    #print(rk)
    profit = int(model.objVal)
    vecteur_z = []
    for v_s in valeurs_scenarios:
        vecteur_z.append(sum([v_s[i,j] for (i,j) in objets_selectiones]))
    return {'vecteur_z':vecteur_z, 'objets': objets_selectiones, 'profit': profit}



In [24]:
for i, couts_instances in enumerate([couts_i2]):
    i = 1
    for k in [2,4,8,16]:
        print(f"------------k = {k}--------------")
        print(f"Plus court chemin de l'instance {i} : ")
        gurobi_solution = minOWA(valeurs_scenarios=couts_instances, w=[k,1])
        print('Chemin: ', gurobi_solution['objets'], ', min_regret: ', gurobi_solution['profit'], ', vecteur_z: ', gurobi_solution['vecteur_z'])
        print("--------------------------------------------------------------------------------------------------\n")

------------k = 2--------------
Plus court chemin de l'instance 1 : 
Chemin:  [('a', 'b'), ('b', 'e'), ('e', 'g')] , min_regret:  30 , vecteur_z:  [10, 10]
--------------------------------------------------------------------------------------------------

------------k = 4--------------
Plus court chemin de l'instance 1 : 
Chemin:  [('a', 'b'), ('b', 'e'), ('e', 'g')] , min_regret:  50 , vecteur_z:  [10, 10]
--------------------------------------------------------------------------------------------------

------------k = 8--------------
Plus court chemin de l'instance 1 : 
Chemin:  [('a', 'b'), ('b', 'e'), ('e', 'g')] , min_regret:  90 , vecteur_z:  [10, 10]
--------------------------------------------------------------------------------------------------

------------k = 16--------------
Plus court chemin de l'instance 1 : 
Chemin:  [('a', 'b'), ('b', 'e'), ('e', 'g')] , min_regret:  170 , vecteur_z:  [10, 10]
--------------------------------------------------------------------------

## minOWA des regrets

\begin{align*}
\min &\sum_{k=1}^n w'_k \left( k \cdot r_k + \sum_{i=1}^n b_{ik} \right) \\
\text{s.c.} & \quad r_k + b_{ik} \geq r(x,s_{(i)}), \quad i = 1, \dots, n \\
& \quad b_{ik} \geq 0, \quad i = 1, \dots, n \\
& \quad x \in X
\end{align*}

In [25]:
def minOWAregrets(valeurs_scenarios,w):

    # trouver les meilleures solutions pour chaque scénario
    z_etoile = []
    for v_s in valeurs_scenarios:
        gurobi_solution = minFlowMaxMin([v_s]) # minFlowMaxMin() avec un scénario est égal à optimiser le problème sad
        z_etoile.append(gurobi_solution['profit'])

    print("z*=", z_etoile)


    model = Model()
    model.params.LogToConsole = False
    # trouver noeuds
    # on suppose que tous les scénarios ont les mêmes arcs
    noeuds = set()
    for arc in valeurs_scenarios[0]:
        noeuds.update(arc)
    noeuds = list(noeuds)

    source = min(noeuds)
    puits = max(noeuds)

    n_scenarios = len(valeurs_scenarios)

    f = model.addVars(valeurs_scenarios[0].keys(), vtype = GRB.BINARY, name='f')

    #w = [i+1 for i in range(n_scenarios)][::-1]
    w_apostrophe = [w[k] - w[k+1] for k in range(len(w)-1)]
    w_apostrophe.append(w[-1])
    #print(w_apostrophe)
    rk = model.addVars(n_scenarios,vtype=GRB.CONTINUOUS, name='rk')
    bik = []
    
    #n**2 variables b 
    for k in range(n_scenarios):
        bik.append(model.addVars(n_scenarios,vtype=GRB.CONTINUOUS, lb=0, name=f'bik {k}'))
    objectif = quicksum([w_apostrophe[k]*((k+1)*rk[k]+quicksum([bik[k][i] for i in range(n_scenarios)])) for k in range(n_scenarios)])
    model.update()
    #print(objectif)
    model.setObjective(objectif, GRB.MINIMIZE)
    
    # calculer vecteur z
    z = []
    for couts in valeurs_scenarios:
        z.append(quicksum([f[i,j] * cout for (i,j),cout in couts.items()]))
    model.update()
   ## print("vecteur z")
    #print(z)
    #print("----------")
    for k in range(n_scenarios):
        for i in range(n_scenarios):
            model.addConstr(rk[k]+bik[k][i]>=z[i]-z_etoile[i]) # premiere contrainte

    # contrainte de conservation du flux
    for v in noeuds:
        if v == source or v == puits:
            continue
        flux_entrant = quicksum([f[i,v] for i,j in valeurs_scenarios[0] if j == v])
        flux_sortant = quicksum([f[v,j] for i,j in valeurs_scenarios[0] if i == v])
        c_cf = flux_entrant == flux_sortant
        model.update()
        #print(c_cf)
        model.addConstr(c_cf)

    # contrainte de flux sortant de la source
    c_fs = quicksum([f[source, j] for i,j in valeurs_scenarios[0] if i == source]) >= 1
    model.update()
   # print(c_fs)
    model.addConstr(c_fs)
    model.optimize()
    
    objets_selectiones = [(i,j) for i,j in valeurs_scenarios[1] if f[i,j].X > 0.5]
    #for b_array in bik:
       # print(b_array)

   # print(rk)
    profit = int(model.objVal)
    vecteur_z = []
    for v_s in valeurs_scenarios:
        vecteur_z.append(sum([v_s[i,j] for (i,j) in objets_selectiones]))
    return {'vecteur_z':vecteur_z, 'objets': objets_selectiones, 'profit': profit}



In [28]:
for i, couts_instances in enumerate([couts_i2]):
    for k in [2,4,8,16]:
        i = 1
        print(f"------------k = {k}--------------")
        print(f"Plus court chemin de l'instance {i} : ")
        gurobi_solution = minOWAregrets(valeurs_scenarios=couts_instances, w=[k,1])
        print('Chemin: ', gurobi_solution['objets'], ', min_regret: ', gurobi_solution['profit'], ', vecteur_z: ', gurobi_solution['vecteur_z'])
        print("--------------------------------------------------------------------------------------------------\n")

------------k = 2--------------
Plus court chemin de l'instance 1 : 
z*= [5, 6]
Chemin:  [('a', 'd'), ('d', 'f'), ('f', 'g')] , min_regret:  13 , vecteur_z:  [6, 12]
--------------------------------------------------------------------------------------------------

------------k = 4--------------
Plus court chemin de l'instance 1 : 
z*= [5, 6]
Chemin:  [('a', 'b'), ('b', 'e'), ('e', 'g')] , min_regret:  24 , vecteur_z:  [10, 10]
--------------------------------------------------------------------------------------------------

------------k = 8--------------
Plus court chemin de l'instance 1 : 
z*= [5, 6]
Chemin:  [('a', 'b'), ('b', 'e'), ('e', 'g')] , min_regret:  44 , vecteur_z:  [10, 10]
--------------------------------------------------------------------------------------------------

------------k = 16--------------
Plus court chemin de l'instance 1 : 
z*= [5, 6]
Chemin:  [('a', 'b'), ('b', 'e'), ('e', 'g')] , min_regret:  84 , vecteur_z:  [10, 10]
--------------------------------